In [17]:
import numpy as np
import pandas as pd
from scipy.stats import wilcoxon
from sklearn.metrics import roc_auc_score, roc_curve
from joblib import Parallel, delayed
import multiprocessing
import time
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from scipy.stats import mannwhitneyu


In [18]:
#Read the Tables
IGB = pd.read_excel("PEP3.xlsx", sheet_name='IGB')
# IGF = pd.read_excel("PLCO2.xlsx", sheet_name='IGF')
CF =pd.read_excel("PEP3.xlsx", sheet_name='CF')

#Fill Na with 0
IGB = IGB.fillna(0)
# IGF = IGF.fillna(0)

In [16]:
# IGF = pd.read_excel('IPAS8000_FT_RAINBOW_EG.Quantity.xlsx').T

In [19]:
PRO = pd.read_excel("20240930_102319_IPAS8000_igB_Rainbow_Integra_EG_Report.xlsx", sheet_name='Mahmood_Transpose').T
PRO2=PRO.iloc[6:,[0,2]]

In [20]:
#Apply the correction Factor
IGB.iloc[:,4:]=IGB.iloc[:,4:].div(CF.CF, axis=0)

In [21]:
#Categorizing the Gene Names

not_gene_columns=['Plate', 'Cancer_Type', 'Is_Case', 'Sex']
k=len(not_gene_columns)

#1-All Genes that we observed in both IGB and IGF
all_genes= list(set(IGB.columns)- set(not_gene_columns))

#2-Mutual Genes that we observed in both IGB and IGF
# mutual_genes= list(set(IGB.columns) & set(IGF.columns) - set(not_gene_columns) ) 

#3-Genes that are only observed in IGF
# only_free = list(set(IGF.columns) - set(IGB.columns)  ) 

#4-Genes that are only observed in IGB
# only_bind = list(set(IGB.columns) - set(IGF.columns)  )

#5-All the genes that we observed in IGB
# all_bind= list(set(IGB.columns)- set(not_gene_columns))

#6-All the genes that we observed in IGF
# all_free=list(set(IGF.columns)- set(not_gene_columns))

In [22]:
#Naming Healthy and Cancer Types
Healthy_Group=['MERIT','LEAP','Benign','All','Other_Cancers']
Cancer_Group=["All","Lung","PDAC","GI","Breast","Ovarian","Rectal"]

In [23]:
# Look at the number of  Cases and Controls based on Sex
print('Control Women =',len(IGB[(IGB.Is_Case==0) & (IGB.Sex==0)]),'Control Men',len(IGB[(IGB.Is_Case==0) & (IGB.Sex==1)]) )

print('Cases Women =',len(IGB[(IGB.Is_Case==0) & (IGB.Sex==0)]),'Cases Men',len(IGB[(IGB.Is_Case==0) & (IGB.Sex==1)]) )

Control Women = 108 Control Men 93
Cases Women = 108 Cases Men 93


In [24]:
def marker_performance(df, type='Is_Case', group_column='Cancer_Type', c='All',
                       h='All', marker='LRG1', gender_column='Sex'):
    """
    This function analyzes the performance of a marker for cancer vs healthy groups.

    Args:
        df (pandas.DataFrame): The input data frame.
        type (str, optional): Column name indicating case/control status. Defaults to 'Is_Case'.
        group_column (str, optional): Column name indicating cancer type. Defaults to 'Cancer_Type'.
        c (str, optional): Specific cancer group to analyze. Defaults to 'All'.
        h (str, optional): Specific healthy group to analyze. Defaults to 'All'.
        marker (str, optional): Marker name to analyze. Defaults to 'LRG1'.
        gender_column (str, optional): Column name indicating gender (if applicable). Defaults to 'Sex'.

    Returns:
        pandas.DataFrame: A DataFrame containing performance metrics.
    """

    df_case = df[df[type] == 1]  # Filter cases
    if c != 'All':
        df_case = df_case[df_case[group_column] == c]  # Filter cases by cancer type (if specified)   
        
    df_control = df[df[type] == 0]  # Filter controls
    if c in ['Breast', 'Ovarian']:  # Filter controls based on gender for Breast/Ovarian cancer
        df_control = df_control[df_control[gender_column] == 0]

    if h != 'All':
        if h in ['Benign', 'MERIT','LEAP']:
            df_control = df_control[df_control[group_column] == h]  # Filter controls by healthy group
        elif h == 'Other_Cancers':
            df_control = df.loc[df[type] == 1]  # Use all cases as controls
            if c == 'All':
                df_control = df.loc[df[type] == 0]  # Use all controls if 'c' is All
            else:
                if c in ['Lung', 'PDAC', 'Rectal', 'GI']:
                    df_control.loc[:, type] = 0  # Set type to 0 for non-matching cancer type
                    df_control = df_control[df_control[group_column] != c]  # Filter controls by non-matching cancer type
                elif c in ['Breast', 'Ovarian']:
                    df_control.loc[:, type] = 0  # Set type to 0 for non-matching cancer type
                    df_control = df_control[df_control[group_column] != c]  # Filter controls by non-matching cancer type
                    df_control = df_control[df_control[gender_column] == 0]  # Filter controls based on gender for Breast/Ovarian
     # Combine cases and controls
    new_df = pd.concat([df_case, df_control])
   
   
        
    # Check for missing values or no data
    if ( (sum(new_df.loc[new_df[type] == 0, marker]!=0)<2) or (sum(new_df.loc[new_df[type] == 1, marker]!=0)<2)):
        df_temp = pd.DataFrame({
            'Marker': marker,
            'Cancer_Type': c,
            'Healthy_Group': h,
            'N_Cases': len(df_case),
            'Percent_Cases': 0.0,
            'N_Controls': len(df_control),
            'Percent_Controls': 0.0,
            'FC_C_to_H': None,
            'AUC': None,
            'Pval': None,
            'Sen_at_95_Spe': None,
            'Spe_at_95_Sen':None

        }, index=[0])
    else:
        # Perform Wilcoxon test
        w_test = mannwhitneyu(new_df.loc[new_df[type] == 1, marker], new_df.loc[new_df[type] == 0, marker])

        # Calculate ROC-AUC and CI
        fpr, tpr, thresholds = roc_curve(new_df[type], new_df[marker])
        auc = roc_auc_score(new_df[type], new_df[marker])
        #ci_lower, ci_upper = auc * np.expm1(-np.linspace(0, 0.05, 100))[:, None]
        

        # Calculate sensitivity at 95% specificity
        sen_at_95_spe=  100 * round(tpr[ np.argmin((1-fpr)>0.95)],4)
        spe_at_95_sen = 100 * round(1-(fpr[ np.argmax(tpr>0.95)]),4)
        
        # sen_at_95_spe =  100 * round(max(tpr[fpr <= 0.05]), 4)
        # spe_at_95_sen = 100 * round(max(fpr[tpr >= 0.95]), 4)

    # Create the data frame
        df_temp = pd.DataFrame({
        'Marker': marker,
        'Cancer_Type': c,
        'Healthy_Group': h,
        'N_Cases': len(df_case),
        'Percent_Cases': round(((len(df_case[df_case[marker] != 0]) / len(df_case)) * 100), 2),
        'N_Controls': len(df_control),
        'Percent_Controls': round(((len(df_control[df_control[marker] != 0]) / len(df_control)) * 100), 2),
        'FC_C_to_H': round(np.mean(df_case[marker]) / np.mean(df_control[marker]), 2),
        'AUC': 100 * round(auc, 4),
        'Pval': w_test.pvalue,
        'Sen_at_95_Spe': sen_at_95_spe,
        'Spe_at_95_Sen': spe_at_95_sen

        }, index=[0])
    return df_temp



In [ ]:
num_cores = multiprocessing.cpu_count()  # Get the number of available cores
from time import time
begin=time()
with Parallel(n_jobs=num_cores) as parallel:
    performance = parallel(delayed(marker_performance)(my_df, c=can, h=heal, marker=marker)
                            for can in Cancer_Group
                            for heal in Healthy_Group
                            for marker in all_genes)
    
performance = pd.concat(performance, ignore_index=True)
performance['CIT']=performance['Marker'].str.contains('Citrullination')
A = pd.merge(performance,PRO2,how='left',left_on='Marker',right_on=2).drop([2], axis=1)
A.to_csv('rainbow_peptide_matched_by_protein_igb_performance_normalized_November_4.csv')#, index=False)
    # performance.to_csv('rainbow_peptide_igb_performance_normalized.csv')#, index=False)
print("Plate ",pl, " Total time:",time()-begin)

In [15]:
for pl in plate_list:
    my_df=IGB[IGB['Plate'].str.contains(pl)]
    num_cores = multiprocessing.cpu_count()  # Get the number of available cores
    from time import time
    begin=time()
    with Parallel(n_jobs=num_cores) as parallel:
        performance = parallel(delayed(marker_performance)(my_df, c=can, h=heal, marker=marker)
                               for can in Cancer_Group
                               for marker in all_genes
                               for heal in Healthy_Group)
    
    performance = pd.concat(performance, ignore_index=True)
    performance['CIT']=performance['Marker'].str.contains('Citrullination')
    A = pd.merge(performance,PRO2,how='left',left_on='Marker',right_on=2).drop([2], axis=1)
    A.to_csv('plate_'+pl+'_rainbow_peptide_matched_by_protein_igb_performance_normalized.csv')#, index=False)
    # performance.to_csv('rainbow_peptide_igb_performance_normalized.csv')#, index=False)
    print("Plate ",pl, " Total time:",time()-begin)

Plate  PL380  Total time: 1501.0038290023804
Plate  PL379  Total time: 1539.2291824817657
Plate  PL378  Total time: 2196.526225090027
Plate  PL377  Total time: 1860.871827363968
Plate  PL376  Total time: 1669.6252551078796
Plate  PL375  Total time: 1645.1810479164124


In [51]:
# for can in Cancer_Group:
#     for heal in Healthy_Group:
#         for marker in all_genes:
#             print(can,heal,marker)
#             marker_performance(IGB, c=can, h=heal, marker=marker)
                          

In [53]:
performance

,Marker,Cancer_Type,Healthy_Group,N_Cases,Percent_Cases,N_Controls,Percent_Controls,FC_C_to_H,AUC,Pval,Sen_at_95_Spe,Spe_at_95_Sen
0,_ELFHPYAESLVSGIGR_.3,All,MERIT,204,21.08,65,29.23,0.68,45.63,0.151139,4.41,0.0
1,_ELFHPYAESLVSGIGR_.3,All,LEAP,204,21.08,99,16.16,1.11,52.30,0.349046,6.37,0.0
2,_ELFHPYAESLVSGIGR_.3,All,Benign,204,21.08,37,16.22,1.89,53.09,0.395667,12.25,0.0
3,_ELFHPYAESLVSGIGR_.3,All,All,204,21.08,201,20.40,0.99,50.29,0.888039,6.37,0.0
4,_ELFHPYAESLVSGIGR_.3,All,Other_Cancers,204,21.08,201,20.40,0.99,50.29,0.888039,6.37,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
375405,_ITDNM[Oxidation (M)]FC[Carbamidomethyl (C)]AG...,Rectal,MERIT,30,83.33,65,89.23,1.04,51.82,0.779090,6.67,0.0
375406,_ITDNM[Oxidation (M)]FC[Carbamidomethyl (C)]AG...,Rectal,LEAP,30,83.33,99,74.75,1.16,60.56,0.079084,0.00,0.0
375407,_ITDNM[Oxidation (M)]FC[Carbamidomethyl (C)]AG...,Rectal,Benign,30,83.33,37,83.78,1.11,56.76,0.346486,3.33,0.0
375408,_ITDNM[Oxidation (M)]FC[Carbamidomethyl (C)]AG...,Rectal,All,30,83.33,201,81.09,1.11,57.03,0.213367,6.67,0.0
